# PatchCoreViT Analysis

## Preparations

In [ ]:
! pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=174e9f1c380e6834a9b0643b638e7d2283efeb548b1ec82849d2374d13126ec1
  Stored in directory: /root/.cache/pip/wheels/01/46/3b/e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget


In [ ]:
import importlib

import os
import cv2
import wget
import json
import torch
import shutil
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

import patchcore_models
import patchcore_utils

from patchcore_utils import get_results, print_results, save_json
from patchcore_models import MVTecDataset, PatchCoreViT, VanillaPatchCore, PatchCoreSWin

class_links = {
    "bottle": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937370-1629951468/bottle.tar.xz",
    "cable": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937413-1629951498/cable.tar.xz",
    "capsule": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937454-1629951595/capsule.tar.xz",
    "carpet": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937484-1629951672/carpet.tar.xz",
    "grid": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937487-1629951814/grid.tar.xz",
    "hazelnut": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937545-1629951845/hazelnut.tar.xz",
    "leather": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937607-1629951964/leather.tar.xz",
    "metal_nut": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937637-1629952063/metal_nut.tar.xz",
    "pill": "https://www.mydrive.ch/shares/43421/11a215a5749fcfb75e331ddd5f8e43ee/download/420938129-1629953099/pill.tar.xz",
    "screw": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938130-1629953152/screw.tar.xz",
    "tile": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938133-1629953189/tile.tar.xz",
    "toothbrush": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938134-1629953256/toothbrush.tar.xz",
    "transistor": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938166-1629953277/transistor.tar.xz",
    "wood": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938383-1629953354/wood.tar.xz",
    "zipper": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938385-1629953449/zipper.tar.xz"
}

'\n    "capsule": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937454-1629951595/capsule.tar.xz",\n    "carpet": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937484-1629951672/carpet.tar.xz",\n    "grid": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937487-1629951814/grid.tar.xz",\n    "hazelnut": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937545-1629951845/hazelnut.tar.xz",\n    "leather": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937607-1629951964/leather.tar.xz",\n    "metal_nut": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937637-1629952063/metal_nut.tar.xz",\n    "pill": "https://www.mydrive.ch/shares/43421/11a215a5749fcfb75e331ddd5f8e43ee/download/420938129-1629953099/pill.tar.xz",\n    "screw": "https://www.mydrive.ch/shares/38536/3830184030e49fe747476694

In [ ]:
# Download duration 6m 5s
for class_name in class_links:
  os.mkdir(class_name)
  file_path = wget.download(class_links[class_name], class_name)
  if os.path.exists(file_path):
    # Extract the file if it's a tar.xz file
    if file_path.endswith('.tar.xz'):
      shutil.unpack_archive(file_path, extract_dir=class_name)
      os.remove(file_path)
      print(f"File {class_name} downloaded and extracted successfully.")
    else:
      print(f"Failed to download the file {class_name}.")

File bottle downloaded and extracted successfully.
File cable downloaded and extracted successfully.


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# remove
# shutil.rmtree("bottle")

## Single Layer Analysis

### Layer 0-1-2

In [ ]:
# Layer 0
print(f"Layer: 0")

model_params = {
  "layers" : [0],
  "backbone" : "google/vit-base-patch16-224-in21k",
  "f_coreset" : 0.1
}

results = get_results(PatchCoreViT, c)
print_results(results)
result_json = save_json(results, "pcViT_base-patch16-224-ink21k_l0.json")
# Avg AUC: 0.795 		Total Misclassified: 306

Layer: 0

Class: bottle


Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


[INFO][__init__] Model PatchCore loaded on device: cuda


100%|██████████| 209/209 [00:12<00:00, 17.14it/s]


Start Coreset Subsampling...


...

[INFO][__init__] Model PatchCore loaded on device: cuda


100%|██████████| 240/240 [00:11<00:00, 21.33it/s]


Start Coreset Subsampling...


100%|██████████| 151/151 [00:09<00:00, 16.08it/s]


Val: IMAGE Level ROCAUC: 0.959
Val: PIXEL Level ROCAUC: 0.935
[INFO][evaluate] Image Level ROCAUC: 0.959
[INFO][evaluate] Initial Score Threshold: 4.054 F1Score: 0.949
[INFO][evaluate] Optimal Score Threshold: 3.944 F1Score: 0.971
[INFO][evaluate] Average Inference time with batch_size=1: 0.022s

CLASS BREAKDOWN
ROCAUC: 0.988 		f1_score: 0.976 	bottle
ROCAUC: 0.477 		f1_score: 0.760 	cable
ROCAUC: 0.803 		f1_score: 0.931 	capsule
ROCAUC: 0.525 		f1_score: 0.864 	carpet
ROCAUC: 0.931 		f1_score: 0.922 	grid
ROCAUC: 0.884 		f1_score: 0.851 	hazelnut
ROCAUC: 0.780 		f1_score: 0.854 	leather
ROCAUC: 0.442 		f1_score: 0.894 	metal_nut
ROCAUC: 0.899 		f1_score: 0.951 	pill
ROCAUC: 0.717 		f1_score: 0.866 	screw
ROCAUC: 0.811 		f1_score: 0.865 	tile
ROCAUC: 0.956 		f1_score: 0.933 	toothbrush
ROCAUC: 0.799 		f1_score: 0.721 	transistor
ROCAUC: 0.950 		f1_score: 0.935 	wood
ROCAUC: 0.959 		f1_score: 0.971 	zipper


SUMMARY
Avg AUC: 0.795 		Total Misclassified: 306


In [ ]:
# Layer 1
print(f"Layer: 1")

model_params = {
  "layers" : [1],
  "backbone" : "google/vit-base-patch16-224-in21k",
  "f_coreset" : 0.1
}

results = get_results(PatchCoreViT, model_params)
print_results(results)
result_json = save_json(results, "pcViT_base-patch16-224-ink21k_l1.json")
# Avg AUC: 0.933 		Total Misclassified: 183

Layer: 1

Class: bottle


Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


[INFO][__init__] Model PatchCore loaded on device: cuda


100%|██████████| 209/209 [00:11<00:00, 17.63it/s]


Start Coreset Subsampling...


...

[INFO][__init__] Model PatchCore loaded on device: cuda


100%|██████████| 240/240 [00:11<00:00, 21.72it/s]


Start Coreset Subsampling...


100%|██████████| 151/151 [00:08<00:00, 17.68it/s]


Val: IMAGE Level ROCAUC: 0.918
Val: PIXEL Level ROCAUC: 0.961
[INFO][evaluate] Image Level ROCAUC: 0.918
[INFO][evaluate] Initial Score Threshold: 3.930 F1Score: 0.927
[INFO][evaluate] Optimal Score Threshold: 3.550 F1Score: 0.952
[INFO][evaluate] Average Inference time with batch_size=1: 0.020s

CLASS BREAKDOWN
ROCAUC: 0.999 		f1_score: 0.992 	bottle
ROCAUC: 0.862 		f1_score: 0.834 	cable
ROCAUC: 0.834 		f1_score: 0.939 	capsule
ROCAUC: 0.852 		f1_score: 0.889 	carpet
ROCAUC: 0.985 		f1_score: 0.965 	grid
ROCAUC: 0.985 		f1_score: 0.952 	hazelnut
ROCAUC: 1.000 		f1_score: 1.000 	leather
ROCAUC: 0.973 		f1_score: 0.963 	metal_nut
ROCAUC: 0.914 		f1_score: 0.936 	pill
ROCAUC: 0.843 		f1_score: 0.882 	screw
ROCAUC: 0.935 		f1_score: 0.901 	tile
ROCAUC: 0.975 		f1_score: 0.938 	toothbrush
ROCAUC: 0.956 		f1_score: 0.892 	transistor
ROCAUC: 0.968 		f1_score: 0.951 	wood
ROCAUC: 0.918 		f1_score: 0.952 	zipper


SUMMARY
Avg AUC: 0.933 		Total Misclassified: 183


In [ ]:
# Layer 2
print(f"Layer: 2")

model_params = {
  "layers" : [2],
  "backbone" : "google/vit-base-patch16-224-in21k",
  "f_coreset" : 0.1
}

results = get_results(PatchCoreViT, model_params)
print_results(results)
result_json = save_json(results, "pcViT_base-patch16-224-ink21k_l2.json")
# Avg AUC: 0.959 		Total Misclassified: 116

Layer: 2

Class: bottle


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

...

100%|██████████| 240/240 [00:10<00:00, 22.64it/s]


Start Coreset Subsampling...


100%|██████████| 151/151 [00:08<00:00, 17.71it/s]


Val: IMAGE Level ROCAUC: 0.923
Val: PIXEL Level ROCAUC: 0.939
[INFO][evaluate] Image Level ROCAUC: 0.923
[INFO][evaluate] Initial Score Threshold: 4.043 F1Score: 0.912
[INFO][evaluate] Optimal Score Threshold: 3.833 F1Score: 0.938
[INFO][evaluate] Average Inference time with batch_size=1: 0.019s

CLASS BREAKDOWN
ROCAUC: 0.999 		f1_score: 0.992 	bottle
ROCAUC: 0.967 		f1_score: 0.935 	cable
ROCAUC: 0.897 		f1_score: 0.936 	capsule
ROCAUC: 0.952 		f1_score: 0.936 	carpet
ROCAUC: 0.992 		f1_score: 0.973 	grid
ROCAUC: 1.000 		f1_score: 0.993 	hazelnut
ROCAUC: 1.000 		f1_score: 1.000 	leather
ROCAUC: 0.998 		f1_score: 0.995 	metal_nut
ROCAUC: 0.946 		f1_score: 0.938 	pill
ROCAUC: 0.821 		f1_score: 0.888 	screw
ROCAUC: 1.000 		f1_score: 1.000 	tile
ROCAUC: 0.919 		f1_score: 0.952 	toothbrush
ROCAUC: 0.992 		f1_score: 0.937 	transistor
ROCAUC: 0.985 		f1_score: 0.975 	wood
ROCAUC: 0.923 		f1_score: 0.938 	zipper


IndexError: invalid index to scalar variable.

In [ ]:
# Layer 3
print(f"Layer: 3")

model_params = {
  "layers" : [3],
  "backbone" : "google/vit-base-patch16-224-in21k",
  "f_coreset" : 0.1
}

results = get_results(PatchCoreViT, model_params)
print_results(results)
result_json = save_json(results, "pcViT_base-patch16-224-ink21k_l3.json")
# Avg AUC: 0.962 		Total Misclassified: 98

Layer: 3

Class: bottle


Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


[INFO][__init__] Model PatchCore loaded on device: cuda


100%|██████████| 209/209 [00:11<00:00, 17.61it/s]


Start Coreset Subsampling...


...

100%|██████████| 240/240 [00:10<00:00, 21.89it/s]


Start Coreset Subsampling...


100%|██████████| 151/151 [00:08<00:00, 18.74it/s]


Val: IMAGE Level ROCAUC: 0.929
Val: PIXEL Level ROCAUC: 0.908
[INFO][evaluate] Image Level ROCAUC: 0.929
[INFO][evaluate] Initial Score Threshold: 4.411 F1Score: 0.922
[INFO][evaluate] Optimal Score Threshold: 4.181 F1Score: 0.964
[INFO][evaluate] Average Inference time with batch_size=1: 0.019s

CLASS BREAKDOWN
ROCAUC: 0.999 		f1_score: 0.992 	bottle
ROCAUC: 0.978 		f1_score: 0.957 	cable
ROCAUC: 0.920 		f1_score: 0.947 	capsule
ROCAUC: 0.967 		f1_score: 0.959 	carpet
ROCAUC: 0.988 		f1_score: 0.964 	grid
ROCAUC: 1.000 		f1_score: 1.000 	hazelnut
ROCAUC: 1.000 		f1_score: 1.000 	leather
ROCAUC: 0.999 		f1_score: 0.995 	metal_nut
ROCAUC: 0.956 		f1_score: 0.957 	pill
ROCAUC: 0.799 		f1_score: 0.865 	screw
ROCAUC: 1.000 		f1_score: 1.000 	tile
ROCAUC: 0.911 		f1_score: 0.952 	toothbrush
ROCAUC: 0.994 		f1_score: 0.964 	transistor
ROCAUC: 0.987 		f1_score: 0.974 	wood
ROCAUC: 0.929 		f1_score: 0.964 	zipper

SUMMARY
Avg AUC: 0.962 		Total Misclassified: 98


{'bottle': {'cm': [[19, 1], [0, 63]],
  'prfs': (0.984375, 1.0, 0.9921259842519685, None),
  'auc': np.float64(0.9992063492063492)},
 'cable': {'cm': [[54, 4], [4, 88]],
  'prfs': (0.9565217391304348, 0.9565217391304348, 0.9565217391304348, None),
  'auc': np.float64(0.9778860569715142)},
 'capsule': {'cm': [[12, 11], [1, 108]],
  'prfs': (0.907563025210084, 0.9908256880733946, 0.9473684210526315, None),
  'auc': np.float64(0.9202233745512565)},
 'carpet': {'cm': [[28, 0], [7, 82]],
  'prfs': (1.0, 0.9213483146067416, 0.9590643274853801, None),
  'auc': np.float64(0.9674959871589085)},
 'grid': {'cm': [[20, 1], [3, 54]],
  'prfs': (0.9818181818181818, 0.9473684210526315, 0.9642857142857143, None),
  'auc': np.float64(0.9883040935672514)},
 'hazelnut': {'cm': [[40, 0], [0, 70]],
  'prfs': (1.0, 1.0, 1.0, None),
  'auc': np.float64(1.0)},
 'leather': {'cm': [[32, 0], [0, 92]],
  'prfs': (1.0, 1.0, 1.0, None),
  'auc': np.float64(1.0)},
 'metal_nut': {'cm': [[22, 0], [1, 92]],
  'prfs': (

In [ ]:
# Layer 4
print(f"Layer: 4")

model_params = {
  "layers" : [4],
  "backbone" : "google/vit-base-patch16-224-in21k",
  "f_coreset" : 0.1
}

results = get_results(PatchCoreViT, model_params)
print_results(results)
result_json = save_json(results, "pcViT_base-patch16-224-ink21k_l4.json")

Layer: 4

Class: bottle


Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


[INFO][__init__] Model PatchCore loaded on device: cuda


100%|██████████| 209/209 [00:11<00:00, 17.94it/s]


Start Coreset Subsampling...


...

100%|██████████| 240/240 [00:10<00:00, 23.35it/s]


Start Coreset Subsampling...


100%|██████████| 151/151 [00:08<00:00, 17.84it/s]


Val: IMAGE Level ROCAUC: 0.947
Val: PIXEL Level ROCAUC: 0.880
[INFO][evaluate] Image Level ROCAUC: 0.947
[INFO][evaluate] Initial Score Threshold: 4.895 F1Score: 0.921
[INFO][evaluate] Optimal Score Threshold: 4.345 F1Score: 0.956
[INFO][evaluate] Average Inference time with batch_size=1: 0.019s


UnboundLocalError: cannot access local variable 'avg_auc_img' where it is not associated with a value

In [ ]:
import patchcore_models
import patchcore_utils

importlib.reload(patchcore_utils)
importlib.reload(patchcore_models)

from patchcore_utils import get_results, print_results, save_json
from patchcore_models import MVTecDataset, PatchCoreViT, VanillaPatchCore, PatchCoreSWin

# print_results(results)
# save_json(results, "pcViT_base-patch16-224-ink21k_l2.json")

# Evaluation Development



In [ ]:
importlib.reload(patchcore_utils)
importlib.reload(patchcore_models)

from patchcore_utils import get_results, print_results, save_json
from patchcore_models import MVTecDataset, PatchCoreViT, VanillaPatchCore, PatchCoreSWin

In [ ]:
model_params = {
  "layers" : [4],
  "backbone" : "google/vit-base-patch16-224-in21k",
  "f_coreset" : 0.1
}

base_path = "/content/"
class_name = "bottle"

temp_path = os.path.join(base_path, class_name, class_name) # ex. /content/bottle/bottle
train_path, test_path  = os.path.join(temp_path, "train", "good"), os.path.join(temp_path, "test")
train_paths, test_paths= [train_path], [os.path.join(test_path, path) for path in os.listdir(test_path)]

pcViT = PatchCoreViT(**model_params)
pcViT.fit(train_paths)

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


[INFO][__init__] Model PatchCore loaded on device: cuda


100%|██████████| 209/209 [00:11<00:00, 18.52it/s]


Start Coreset Subsampling...


100%|██████████| 4095/4095 [00:05<00:00, 783.10it/s]


In [ ]:
test_dataloader = pcViT.get_dataloader(test_paths)

for sample_evaluate, label, path, mask in test_dataloader:
    break

sample_ale = pcViT.process_input(path[0])

print(sample_evaluate.pixel_values[0].shape)
print(sample_ale.pixel_values[0].shape)

torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])


In [ ]:
# Layer 4
print(f"Layer: 4")

model_params = {
  "layers" : [4],
  "backbone" : "google/vit-base-patch16-224-in21k",
  "f_coreset" : 0.1
}

results = get_results(PatchCoreViT, model_params)

Layer: 4

Class: bottle


Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


[INFO][__init__] Model PatchCore loaded on device: cuda


100%|██████████| 209/209 [00:11<00:00, 17.99it/s]


Start Coreset Subsampling...


...

[INFO][__init__] Model PatchCore loaded on device: cuda


100%|██████████| 224/224 [00:17<00:00, 12.77it/s]


Start Coreset Subsampling...


100%|██████████| 150/150 [00:13<00:00, 11.25it/s]


Val: IMAGE Level ROCAUC: 0.987
Val: PIXEL Level ROCAUC: 0.981
[INFO][evaluate] Image Level ROCAUC: 0.987
[INFO][evaluate] Initial Score Threshold: 6.128 F1Score: 0.944
[INFO][evaluate] Optimal Score Threshold: 6.038 F1Score: 0.952
[INFO][evaluate] Average Inference time with batch_size=1: 0.025s


In [ ]:
print("Layer 4")
print_results(results)
result_json = save_json(results, "development.json")

Layer 4


CLASS BREAKDOWN
ROCAUC img: 1.000	ROCAUC pxl: 0.988	f1_score: 1.000 	bottle
ROCAUC img: 0.987	ROCAUC pxl: 0.981	f1_score: 0.952 	cable

SUMMARY
Avg ROCAUC img: 0.993
Avg ROCAUC pxl: 0.985
Total Misclassified: 9
